# Exercises transfer learning feature extraction

In [1]:
# Import dependencies
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import zipfile
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import datetime

## Exercise 1: Build and fit a model using the same data of the class but with the MobileNetV2 architecture feature extraction (mobilenet_v2_100_224/feature_vector) from TensorFlow Hub, how does it perform compared to our other models?

### Importing the data

In [ ]:
# Download data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip", "r")
zip_ref.extractall()
zip_ref.close()

--2023-11-03 10:48:08--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.207, 142.251.8.207, 142.251.170.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  30.8MB/s    in 6.0s    

2023-11-03 10:48:14 (26.8 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [ ]:
# How many images in each folder?
# Walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent

### Preparing the data (creating data loaders)

In [ ]:
IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

train_data_10_percent = train_datagen.flow_from_directory(train_dir, target_size=IMAGE_SHAPE,
                                                          batch_size=BATCH_SIZE, class_mode="categorical")

test_data = test_datagen.flow_from_directory(test_dir, target_size=IMAGE_SHAPE,
                                                          batch_size=BATCH_SIZE, class_mode="categorical")

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


### Setting up callbacks

In [ ]:
# Create TensorBoard callback (functionized beause we need to create a new one for each model)
def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"Saving TensorBoard log files to: {log_dir}")
    return tensorboard_callback

### Creating model

In [ ]:
# Let's compare the following two models
resnet_url ="https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"
efficientnet_url ="https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

# IMAGE_SHAPE + (3,)  = (224, 224, 3)

# Let's make a create_model() function to create a model from a URL
def create_model(model_url, num_classes=10):
    """
    Takes a TensorFlow Hub URL and creates a Keras Sequential model with
    it.
    Args:
        model_url (str): A TensorFlow Hub feature extraction URL.
        num_classes (int): Number of output neurons in the output layer,
        should be equal to number of target classes, default 10.
    Returns:
      An uncompiled Keras Sequential model with model_url as feature
      extractor layer and Dense output layer with num_classes output neurons.
    """
    # Download the pretrained model and save it as a Keras layer
    feature_extractor_layer = hub.KerasLayer(model_url, trainable=False, # freeze the already learned patterns
    name="feature_extraction_layer", input_shape=IMAGE_SHAPE+(3,))

    # Create our own model
    model = tf.keras.Sequential([
    feature_extractor_layer,
    layers.Dense(num_classes, activation="softmax",
    name="output_layer")
    ])

    return model

In [ ]:
# Create mobilenet model
mobilenet_model = create_model("https://www.kaggle.com/models/google/mobilenet-v3/frameworks/TensorFlow2/variations/large-075-224-classification/versions/1",
                               num_classes=train_data_10_percent.num_classes)

# Compile our mobilenet model
mobilenet_model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(),
                        metrics=["accuracy"])

# Let's fit our ResNet model to the data (10 percent of 10 classes)
mobilenet_history = mobilenet_model.fit(train_data_10_percent, epochs=5,
                                     steps_per_epoch=len(train_data_10_percent),
                                     validation_data=test_data,
                                     validation_steps=len(test_data),
                                     callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",
                                                                            experiment_name="mobilenetV3")])


Saving TensorBoard log files to: tensorflow_hub/mobilenetV3/20231103-104829
Epoch 1/5
24/24 [==============================] - 23s 451ms/step - loss: 2.2270 - accuracy: 0.3187 - val_loss: 1.2052 - val_accuracy: 0.5972
Epoch 2/5
24/24 [==============================] - 8s 363ms/step - loss: 0.9439 - accuracy: 0.6960 - val_loss: 0.7510 - val_accuracy: 0.7504
Epoch 3/5
24/24 [==============================] - 9s 378ms/step - loss: 0.6180 - accuracy: 0.7987 - val_loss: 0.6194 - val_accuracy: 0.7916
Epoch 4/5
24/24 [==============================] - 9s 393ms/step - loss: 0.4590 - accuracy: 0.8600 - val_loss: 0.5865 - val_accuracy: 0.8052
Epoch 5/5
24/24 [==============================] - 10s 418ms/step - loss: 0.3602 - accuracy: 0.9013 - val_loss: 0.5634 - val_accuracy: 0.8140


In [2]:
# Plots our loss curves... Tidbit: you could put a function like this into a
# script called "helper.py" and import it when you need it...

# Plot the validation and training curves
def plot_loss_curves(history):
    """
    Returns separate loss curves for training and validation metrics.
    Args:
      history: TensorFlow History object.
    Returns:
      Plots of training/validation loss and accuracy metrics.
    """
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    accuracy = history.history["accuracy"]
    val_accuracy = history.history["val_accuracy"]
    epochs = range(len(history.history["loss"]))
    # Plot loss
    plt.plot(epochs, loss, label="training_loss")
    plt.plot(epochs, val_loss, label="val_loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    # Plot accuracy
    plt.figure()
    plt.plot(epochs, accuracy, label="training_accuracy")
    plt.plot(epochs, val_accuracy, label="val_accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.legend();


In [ ]:
plot_loss_curves(mobilenet_history)

## Exercise 2: Name 3 different image classification models on TensorFlow Hub that we haven't used.

1. inception_v3
2. Inception ResNet
3. Inception family of models
4. NASNet models
5. Some of the ResNet versions of models



## Exercise 3: Build a model to classify images of two different things you've taken photos of.

*   You can use any feature extraction layer from TensorFlow Hub you like for this.
* You should aim to have at least 10 images of each class, for example to build a fridge versus oven classifier, you'll want 10 images of fridges and 10 images of ovens.

### Importing the data

In [4]:
# We upload our custom folder and we extract it
zip_ref = zipfile.ZipFile("shoesAndPens.zip", "r")
zip_ref.extractall()
zip_ref.close()

# Walk through data directory and list number of files
for dirpath, dirnames, filenames in os.walk("shoesAndPens"):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in 'shoesAndPens'.
There are 2 directories and 0 images in 'shoesAndPens/test'.
There are 0 directories and 2 images in 'shoesAndPens/test/pens'.
There are 0 directories and 2 images in 'shoesAndPens/test/shoes'.
There are 2 directories and 0 images in 'shoesAndPens/train'.
There are 0 directories and 10 images in 'shoesAndPens/train/pens'.
There are 0 directories and 10 images in 'shoesAndPens/train/shoes'.


### Preparing the data (creating data loaders)

In [6]:
train_dir = "shoesAndPens/train/"
test_dir = "shoesAndPens/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

train_data = train_datagen.flow_from_directory(train_dir, target_size=(224, 224),
                                                          batch_size=8, class_mode="binary")

test_data = test_datagen.flow_from_directory(test_dir, target_size=(224, 224),
                                                          batch_size=8, class_mode="binary")

Found 20 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


### Creating model

In [8]:
efficientnet_feature_extraction_layer = hub.KerasLayer("https://www.kaggle.com/models/google/efficientnet-v2/frameworks/TensorFlow2/variations/imagenet1k-b0-classification/versions/2",
                                                       trainable=False, input_shape=(224, 224) + (3,))


In [9]:
# Building a Sequential model with our mobilenet feature extraction layer for our custom data
custom_data_model = tf.keras.Sequential([
  efficientnet_feature_extraction_layer,
  layers.Dense(1, activation= 'sigmoid', name ='output_layer')
])

# Printing the summarr of the model
custom_data_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1000)              7200312   
                                                                 
 output_layer (Dense)        (None, 1)                 1001      
                                                                 
Total params: 7201313 (27.47 MB)
Trainable params: 1001 (3.91 KB)
Non-trainable params: 7200312 (27.47 MB)
_________________________________________________________________


In [11]:
# Compiling the model
custom_data_model.compile(loss = tf.keras.losses.BinaryCrossentropy() ,
                          optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001) ,
                          metrics = ['accuracy'])


# Fitting the model
custom_model_history = custom_data_model.fit(train_data,
                      epochs=5)

Epoch 1/5
3/3 [==============================] - 3s 375ms/step - loss: 0.0186 - accuracy: 1.0000
Epoch 2/5
3/3 [==============================] - 1s 417ms/step - loss: 0.0097 - accuracy: 1.0000
Epoch 3/5
3/3 [==============================] - 1s 419ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 4/5
3/3 [==============================] - 1s 336ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 5/5
3/3 [==============================] - 1s 430ms/step - loss: 0.0020 - accuracy: 1.0000


## Exercise 4: What is the current best performing model on ImageNet?
Hint: you might want to check sotabench.com for this.

It seems like BASIC-L (Lion, fine-tuned) and efficient-netL2 is the state-of-the-art model now performing really good in Image Classification task.

Top 1% accuracy --> 91.1% (BASIC-L)
Top 5% accuracy --> 98.9% (efficient-netL2)

To know more:

https://paperswithcode.com/sota/image-classification-on-imagenet